## Train Model

The idea of this step is to leverage existing state of the art models such as Ince

# Create image generators

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
from PIL import Image

Using TensorFlow backend.


In [2]:
def generate_generator_multiple(generator,directories, batch_size, img_height,img_width):
    generators =[]
    for directory in directories:
        gen = generator.flow_from_directory(directory,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
        generators.append(gen)

    for gen in generators:
        for data, labels in gen:
            yield data, labels

In [3]:
def save_plots(history,target_file_acc,target_file_loss):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_acc)
    plt.close()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_loss)
    plt.close()

In [4]:
def show_plots(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Doing K-Folds

In [5]:
batch_size = 32
img_height=128
img_width = 256
approx_fold_size = 800

In [6]:
urban_sounds_folder = "/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms"

In [7]:
fold_directories = []
for i in range(1,11):
    directory = urban_sounds_folder+"/fold"+str(i)
    fold_directories.append(directory)

In [8]:
fold_directories

['/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold1',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold2',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold3',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold4',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold5',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold6',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold7',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold8',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold9',
 '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/spectrograms/fold10']

# MODEL

In [9]:
datagen = ImageDataGenerator(rescale=1./255)

In [10]:
input_shape = (img_height, img_width,3)
nclass = 10

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(nclass, activation=tf.nn.softmax))

In [13]:
opt = optimizers.SGD(lr=0.05, momentum=0.9,decay = 0.002)

In [14]:
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])
#needed to reset weights!
model.save_weights('raw_model.h5')
model.summary()

InternalError: GPU sync failed

# Folding

In [ ]:
num_folds = 10 #1 to 10
fold = 0
for directory in fold_directories:
    fold +=1
    #RESET WEIGHTS!!
    model.load_weights('raw_model.h5')
    #
    train_directories = list(set(fold_directories) - set([directory]))
    test_directories = [directory]
    print("Running fold {}, holding data from {} and training on the remaining {}" \
          .format(fold,directory,len(train_directories)))
    train_generator = generate_generator_multiple(generator=datagen,
                                           directories = train_directories,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)
    test_generator = generate_generator_multiple(generator=datagen,
                                       directories = test_directories,
                                       batch_size=batch_size,
                                       img_height=img_height,
                                       img_width=img_width)
    history = model.fit_generator(train_generator,
                              steps_per_epoch=approx_fold_size*9/batch_size,
                              epochs=100,
                              validation_data = test_generator,
                              validation_steps=approx_fold_size/batch_size,
                              shuffle=True, 
                              verbose=True)
    model.save_weights('trained_model_fold_{}.h5'.format(fold))
    with open('training_history_fold_{}.json'.format(fold), 'w') as f:
        json.dump(history.history, f)
    save_plots(history,'training_accuracy_plot_fold_{}.png'.format(fold),
               'training_lossaccuracy_plot_fold_{}.png'.format(fold))
    if fold == num_folds:
        break

In [ ]:
show_plots(history)